<a href="https://colab.research.google.com/github/Untick/resume-builder-gr2/blob/Labzin-Vyacheslav-branch/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%B7%D0%B0%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 unstructured pdf2image pdfminer.six faiss-cpu==1.7.4

In [1]:
!pip install openai==0.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import openai
import getpass
import re
import requests
import os
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [4]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
# Функция для представления длинного ответа в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    """
    Функция разбивает длинный текст на строки определенной максимальной длины.
    """
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [17]:
# ПРОМПТ в system
system = load_document_text('https://docs.google.com/document/d/12fSAEOmhbkLTHrDq84OCI8Myo0HnHQ575yh02lzCNI8/edit?usp=sharing')

In [7]:
# Образец резюме
resume_sample = load_document_text(
    'https://docs.google.com/document/d/1UeR8y8husKJCVT5M-mQ_N8rKk-BKVAex/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [8]:
# Резюме без образования
resume_none_educ = load_document_text(
    'https://docs.google.com/document/d/1qBuuFUhOPLqpIhfX5-ZdrhN8YVtzOrF3/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [9]:
# Резюме без контактов
resume_none_cont = load_document_text(
    'https://docs.google.com/document/d/1eUx1vIDmLk1frN2h8mHTiRkDY7hqgjc0/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [10]:
# Резюме без навыков
resume_none_skills = load_document_text(
    'https://docs.google.com/document/d/1itZcIdHrSe9_SuheJvOx7Qm084WFfEW3/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [22]:
# Другое резюме без навыков
resume_none_skills_2 = load_document_text(
    'https://docs.google.com/document/d/1IdeQdCB_Yqubn4UDpVGTHEK0IrcKiTey/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [11]:
# Резюме с нерелевантным образованием
resume_fall_educ = load_document_text(
    'https://docs.google.com/document/d/1vdIIE7dfBtfDvi3ZjzU-2bOSfLe6Lkq_/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [12]:
# Резюме без блока "О себе"
resume_none_me = load_document_text(
    'https://docs.google.com/document/d/1Ije4qmt7Nur1E4zzrIoCWE442X8xOsVf/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [20]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_sample}"},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_cont}."}
    ]
)
answer = insert_newlines(response['choices'][0]['message']['content'])
print(answer)

 - В блоке "КОНТАКТЫ" не указаны контактные данные, такие как телефон и email. - В блоке "ОБО МНЕ" не указано, что ты являешься выпускником IT университета. - Блок
 "НАВЫКИ" заполнен правильно. - В блоке "ПРОФЕССИОНАЛЬНАЯ ДЕЯТЕЛЬНОСТЬ" указаны нерелевантные для IT специалиста позиции. - В блоке "ОБРАЗОВАНИЕ" отсутствуют данные о
 твоем образовании в IT сфере.


In [18]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_sample}"},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_me}."}
    ]
)

answer = insert_newlines(response['choices'][0]['message']['content'])
print(answer)

 В резюме не заполнено поле "О СЕБЕ". Добавьте информацию о себе, связанную с IT специальностью, например, ваш опыт работы с анализом данных и разработкой нейронных
 сетей. Также, блок "Навыки" заполнен некорректно. Укажите конкретные IT навыки, такие как Python, GPT (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn,
 Keras, TensorFlow.


In [25]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_sample}"},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_fall_educ}."}
    ]
)

answer = insert_newlines(response['choices'][0]['message']['content'])
print(answer)

 Замечания по заполнению резюме: 1. В блоке "О себе" указано, что вы являетесь Junior специалистом в сфере искусственного интеллекта и имеете опыт работы с анализом
 данных и разработкой нейронных сетей. Это информация релевантна и соответствует IT специальности. 2. В блоке "Навыки" указаны навыки, связанные с IT: Python, GPT
 (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn, Keras, TensorFlow. Навыки соответствуют IT специальности. 3. В блоке "Профессиональная деятельность"
 указаны проекты и стажировки, связанные с IT и анализом данных. Это релевантная информация. 4. В блоке "Образование" указано ваше высшее образование, связанное с
 инженерно-технической специальностью. Возможно, стоит добавить информацию о получении образования в области IT или специализированных курсах. Основные замечания к
 резюме: добавить информацию об образовании, связанном с IT, и более подробно описать проекты и стажировки, связанные с анализом данных и разработкой нейронных сетей.


In [19]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_sample}"},
        {"role": "user", "content": f" Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_skills}."}
    ]
)

answer = insert_newlines(response['choices'][0]['message']['content'])
print(answer)

 - В блоке "Навыки" не указаны конкретные IT навыки. Добавьте IT навыки, такие как Python, GPT (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn, Keras,
 Tensorflow. - В блоке "Образование" не указано связанное с IT специальностью образование. Добавьте информацию о вашем образовании, связанном с Data Science и нейронными
 сетями, например, курсы, сертификаты или программы обучения. - В блоках "Профессиональная деятельность" указаны опыт работы, не связанный с IT специальностью. Уберите
 информацию о работе, которая не относится к IT сфере. - В блоке "О себе" исчезло описание вашего опыта работы с анализом данных, разработкой нейронных сетей и активного
 изучения и работы с ChatGPT. Добавьте эту информацию в блок "О себе".


In [23]:
# Изменил резюме
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_sample}"},
        {"role": "user", "content": f" Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_skills_2}."}
    ]
)

answer = insert_newlines(response['choices'][0]['message']['content'])
print(answer)

 Замечания по заполнению резюме: 1. В блоке "Навыки" не указаны IT навыки. Укажите конкретные IT навыки, которыми вы владеете, например: Python, C++, Java, NumPy, Pandas,
 Matplotlib, Seaborn, Librosa, Gensim, Pymorphy2, Scikit-learn, SciPy, Keras, Pytorch. 2. В блоке "Профессиональная деятельность" необходимо указать должности и задачи,
 связанные с IT сферой. Удалите информацию о приеме сотрудников на работу в ООО "РЖД" и формировании бюджета в ОАО "ГАЗ", так как они не относятся к IT специализации. 3.
 В блоке "Образование" не указано образование, связанное с IT специальностями. Укажите ваше образование в сфере искусственного интеллекта или data science.
